In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

In [19]:
df=pd.read_csv('https://raw.githubusercontent.com/Fangda2023/schulich_data_science/main/experiment_dataset.csv')

In [6]:
df

,Unnamed: 0,Age,Location,Device,Variant,Time Spent,CTR
0,0,62,Location2,Device2,Control,13.928669,0.084776
1,1,18,Location1,Device1,Variant B,11.310518,0.096859
2,2,21,Location2,Device1,Variant B,24.842100,0.097630
3,3,21,Location1,Device3,Variant B,20.061300,0.109783
4,4,57,Location1,Device2,Variant B,34.495503,0.068579
...,...,...,...,...,...,...,...
995,995,39,Location2,Device2,Variant B,17.252030,0.092211
996,996,38,Location3,Device2,Control,30.075898,0.078151
997,997,60,Location2,Device3,Control,31.929223,0.125213
998,998,35,Location2,Device2,Variant B,14.680299,0.095423


In [4]:
df.describe()

,Unnamed: 0,Age,Time Spent,CTR
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,40.715000,22.713345,0.109145
std,288.819436,13.655557,5.479292,0.022366
min,0.000000,18.000000,7.114806,0.012975
25%,249.750000,28.000000,19.216608,0.094286
50%,499.500000,41.000000,22.506707,0.108944
75%,749.250000,53.000000,26.255950,0.124238
max,999.000000,64.000000,39.395770,0.172728


In [20]:
#set index by adding 1 to all numbers in Unnamed:0
df['Unnamed: 0']=df['Unnamed: 0']+1
df=df.rename(columns={'Unnamed: 0':'customer_id'})
df.set_index('customer_id', inplace=True)


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         1000 non-null   int64  
 1   Location    1000 non-null   object 
 2   Device      1000 non-null   object 
 3   Variant     1000 non-null   object 
 4   Time Spent  1000 non-null   float64
 5   CTR         1000 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 54.7+ KB


#### 1.Analyze the results to determine which feature (if any) results in CTR or Time Spent lift.

##### test first if different location and device will impact the CTR and Time spent.

In [35]:
# ANOVA test to see if the difference in CTR is statistically significant.
loc1 = df[df['Location'] == 'Location1']['CTR']
loc2 = df[df['Location'] == 'Location2']['CTR']
loc3 = df[df['Location'] == 'Location3']['CTR']

p_val = stats.f_oneway(loc1, loc2, loc3)
print(p_val)

F_onewayResult(statistic=0.5792245145655729, pvalue=0.5605211716238133)


In [36]:
# ANOVA test to see if the difference in Time Spent is statistically significant.
loc1 = df[df['Location'] == 'Location1']['Time Spent']
loc2 = df[df['Location'] == 'Location2']['Time Spent']
loc3 = df[df['Location'] == 'Location3']['Time Spent']

p_val = stats.f_oneway(loc1, loc2, loc3)
print(p_val)

F_onewayResult(statistic=0.053578839670573676, pvalue=0.9478339402848069)


there is no statistically significant difference between the CTR and time spent of users from different locations.

In [37]:
# ANOVA test to see if the difference in CTR is statistically significant.
dev1 = df[df['Device'] == 'Device1']['CTR']
dev2 = df[df['Device'] == 'Device2']['CTR']
dev3 = df[df['Device'] == 'Device3']['CTR']

p_val = stats.f_oneway(dev1, dev2, dev3)
print(p_val)

F_onewayResult(statistic=0.7105872492654717, pvalue=0.4916042399968955)


In [38]:
# ANOVA test to see if the difference in Time Spent is statistically significant.
dev1 = df[df['Device'] == 'Device1']['Time Spent']
dev2 = df[df['Device'] == 'Device2']['Time Spent']
dev3 = df[df['Device'] == 'Device3']['Time Spent']

p_val = stats.f_oneway(dev1, dev2, dev3)
print(p_val)

F_onewayResult(statistic=0.26645371811833884, pvalue=0.7661459958744103)


there is no statistically significant difference between the CTR and time spent of users with different Devices.

In [24]:
#compare the CTR and TIME SPENT of the 2 variant from the original feature(the control)
df.groupby('Variant')[['CTR','Time Spent']].mean()

,CTR,Time Spent
Variant,,
Control,0.098554,20.070781
Variant A,0.120269,24.805547
Variant B,0.108933,23.343783


both variants increase the CTR and Time spent on average, while Variant A seems to have a better performance on both metrics.

#### 2. Conduct statistical testing to determine if there is a statistically significant difference between the features and the control group.

In [30]:
# ANOVA test to see if the difference in CTR is statistically significant.
ctrl_ctr = df[df['Variant'] == 'Control']['CTR']
VarA_ctr = df[df['Variant'] == 'Variant A']['CTR']
VarB_ctr = df[df['Variant'] == 'Variant B']['CTR']

p_val = stats.f_oneway(ctrl_ctr, VarA_ctr, VarB_ctr)
print(p_val)

F_onewayResult(statistic=93.588915936227, pvalue=5.638952705781955e-38)


since the p-value is much smaller than 0,05, we can reject the null hypothesis and conclude that at least one group of feature is different from the others.

In [31]:
# t-test to see if the difference between the origin design and the 2 variants
p_val = stats.ttest_ind(VarA_ctr, ctrl_ctr)
p_val_2 = stats.ttest_ind(VarB_ctr, ctrl_ctr)
p_val_3 = stats.ttest_ind(VarB_ctr, VarA_ctr)
print(p_val)
print(p_val_2)
print(p_val_3)

Ttest_indResult(statistic=13.829424737499187, pvalue=1.9602781373243157e-38)
Ttest_indResult(statistic=6.4718143491783255, pvalue=1.8743198199982106e-10)
Ttest_indResult(statistic=-7.08499696316128, pvalue=3.587180487986577e-12)


The t-tests show statistically significant improvements from both variant A and B, and that variant A is better than B in terms of CTR.

Now test if there is difference in users time spent between the origin design and the 2 variants

In [33]:
ctrl_ts = df[df['Variant'] == 'Control']['Time Spent']
VarA_ts = df[df['Variant'] == 'Variant A']['Time Spent']
VarB_ts = df[df['Variant'] == 'Variant B']['Time Spent']
p_val = stats.ttest_ind(VarA_ts, ctrl_ts)
p_val_2 = stats.ttest_ind(VarB_ts, ctrl_ts)
p_val_3 = stats.ttest_ind(VarB_ts, VarA_ts)
print(p_val)
print(p_val_2)
print(p_val_3)

Ttest_indResult(statistic=12.142363487472364, pvalue=8.488565644996449e-31)
Ttest_indResult(statistic=8.174237395991806, pvalue=1.496358076285182e-15)
Ttest_indResult(statistic=-3.6788175394209075, pvalue=0.0002534771014765265)


The t-tests also show statistically significant improvements in users time spent from both variant A and B, and that variant A is still better than B.